In [1]:
import pymysql

In [ ]:
conn = pymysql.connect(host = '127.0.0.1', user='root',password='1234', charset='utf8')

cursor = conn.cursor()

In [8]:
sql = 'create database if not exists flask_db;'
cursor.execute(sql)

cursor.execute('show databases;')

result = cursor.fetchall()

print(result)

(('information_schema',), ('budongsan',), ('employees',), ('fin_db',), ('flask_db',), ('gmarketdb',), ('mysql',), ('performance_schema',), ('sakila',), ('shopdb',), ('sqldb',), ('sys',), ('tabledb',), ('world',))


In [9]:
cursor.execute('use flask_db')

0

In [12]:
sql = """
create table if not exists member (
`id` int(11) not null auto_increment,
`userid` varchar(50) not null,
`pwd` varchar(200) not null,
`name` varchar(20) default null,
`email` varchar(50) default null,
`regdate` datetime default null,
primary key(`id`))
"""

cursor = conn.cursor()
cursor.execute(sql)

0

In [17]:
cursor.execute('show tables;')
result = cursor.fetchall()
result

(('member',),)

In [18]:
sql = """
insert into member values 
(1, 'aaa', '1234', '아이유', 'aaa@aaa.co.kr', '2019-06-24'),
(2, 'bbb', '1234', '방탄소년', 'bbb@bbb.co.kr', '2019-06-24')
"""

cursor.execute(sql)

2

In [19]:
cursor.execute('select * from member;')
result = cursor.fetchall()
result

((1,
  'aaa',
  '1234',
  '아이유',
  'aaa@aaa.co.kr',
  datetime.datetime(2019, 6, 24, 0, 0)),
 (2,
  'bbb',
  '1234',
  '방탄소년',
  'bbb@bbb.co.kr',
  datetime.datetime(2019, 6, 24, 0, 0)))

In [21]:
sql = """
create table if not exists point_table(
`point_stu_idx` int(11) default null,
`point_stu_grade` varchar(50) default null,
`point_stu_kor` varchar(50) default null
)
"""
cursor.execute(sql)

0

In [22]:
cursor.execute('select * from point_table;')
result = cursor.fetchall()
result

()

In [23]:
sql = """
insert into point_table values
(2, '2', '90'),
(2, '3', '94'),
(1, '1', '89');
"""

cursor.execute(sql)

3

In [24]:
cursor.execute('select * from point_table;')
result = cursor.fetchall()
result

((2, '2', '90'), (2, '3', '94'), (1, '1', '89'))

In [25]:
sql = """
CREATE TABLE IF NOT EXISTS student_table (
  `stu_idx` int(11) NOT NULL AUTO_INCREMENT,
  `stu_name` varchar(50) DEFAULT NULL,
  `stu_age` varchar(3) DEFAULT NULL,
  `stu_addr` varchar(50) DEFAULT NULL,
  PRIMARY KEY (`stu_idx`)
) ;
"""
cursor.execute(sql)


0

In [26]:
sql = """
INSERT INTO student_table (`stu_idx`, `stu_name`, `stu_age`, `stu_addr`) VALUES
   (1, '홍길동', '18', '서울시'),
   (2, '김개똥', '19', '서울시');
"""
cursor.execute(sql)

2

In [27]:
# db 반영

conn.commit()

In [28]:
def get_connection():
    conn = pymysql.connect(host = '127.0.0.1', user='root',password='1234', charset='utf8')
    return conn

In [29]:
conn = get_connection()

In [30]:
cursor = conn.cursor()

In [31]:
cursor.execute('show databases;')

14

In [32]:
cursor.execute('use flask_db')

0

In [33]:
sql = """
select stu_idx, stu_name, stu_age, stu_addr from student_table where stu_name like {}
""".format(name)

cursor.execute(sql)
result = cursor.fetchall()
result

NameError: name 'name' is not defined

In [36]:
def get_student_list():
    sql = """
    select stu_idx, stu_name, stu_age, stu_addr from student_table;
    """
    
    cursor.execute(sql)
    result = cursor.fetchall()
    return result


In [37]:
student_list = get_student_list()
student_list

((1, '홍길동', '18', '서울시'), (2, '김개똥', '19', '서울시'))

In [38]:
def get_student_list(stu_name):
    
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute('use flask_db;')
    
    
    sql = """
    select stu_idx, stu_name, stu_age, stu_addr from student_table
    """
    if stu_name != None and len(stu_name) > 0:
        sql += ' where stu_name like %s'
        
    sql += ' order by stu_idx desc;'
        
 
    if stu_name != None and len(stu_name) > 0:
        cursor.execute(sql, (f'%{stu_name}%'))
    else:
        cursor.execute(sql)
        
    result = cursor.fetchall()
    
    temp_list = []
    
    for row in result :
        temp_dict = {}
        temp_dict['stu_idx'] = row[0]
        temp_dict['stu_name'] = row[1]
        temp_dict['stu_age'] = row[2]
        temp_dict['stu_addr'] = row[3]
        
        temp_list.append(temp_dict)
        
    conn.close()
    
    return temp_list

In [39]:
student_list = get_student_list([])
student_list

[{'stu_idx': 2, 'stu_name': '김개똥', 'stu_age': '19', 'stu_addr': '서울시'},
 {'stu_idx': 1, 'stu_name': '홍길동', 'stu_age': '18', 'stu_addr': '서울시'}]

In [40]:
def add_student(stu_name, stu_age, stu_addr):
    sql = """
    insert into student_table (stu_name, stu_age, stu_addr) values
    (%s, %s, %s) """
    
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute('use flask_db;')    
    
    cursor.execute(sql, (stu_name, stu_age, stu_addr))
    conn.commit()
    
    sql2 = 'select max(stu_idx) from student_table;'
    cursor.execute(sql2)
    result = cursor.fetchone()
    idx = result[0]
    
    conn.close()
    
    return idx

In [41]:
def get_student_info(stu_idx):
    sql = """
    select stu_name, stu_age, stu_addr from student_table where stu_idx=%s
    """
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute('use flask_db;')     
    
    cursor.execute(sql, (stu_idx))
    result = cursor.fetchone()
    
    result_dic = {}
    result_dic['stu_idx'] = stu_idx
    result_dic['stu_name'] = result[0]
    result_dic['stu_age'] = result[1]
    result_dic['stu_addr'] = result[2]
    
    conn.close()
    
    return result_dic

In [42]:
def add_point(point_stu_idx, point_stu_grade, point_stu_kor):
    
    sql = """
    insert into point_table values (%s, %s, %s)
    """
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute('use flask_db;')     
    
    cursor.execute(sql, (point_stu_idx, point_stu_grade, point_stu_kor))
    
    conn.commit()
    
    conn.close()